In [1]:
import json
import sqlite3
import pandas as pd
import seaborn as sns

In [2]:
con = sqlite3.connect('../data/interim/articles_with_author_mapping.db')
cur = con.cursor()

In [3]:
cur.execute("select id, name, abbreviation, matching_certainty from authors")
rows = cur.fetchall()
authors = pd.DataFrame(columns=["id", "name", "abbreviation", "certainty"], data=rows)
authors.set_index("id", inplace=True)

### Potential Final mapping of an abbreviation to a name
Test if significant shares of an abbreviation to same name exist. If theres is a dominant share, assign this as a final mapping

In [16]:
def mapping_func(abbr):
    abbr = abbr_mapping_func(abbr)
    return abbr

def name_mapping_func(name):
    print("Name: " + name)
    # get all names pointing to that abbreviation
    abbreviations = authors[authors.name == name].abbreviation
    certainties = authors[authors.name == name][["abbreviation", "certainty"]].groupby("abbreviation").agg({"certainty": "mean"}).reset_index()
    frequencies = abbreviations.value_counts(normalize=True)
    # merge certainties and frequencies on name
    merged = pd.merge(certainties, frequencies, on="abbreviation")
    print(merged)
    return name

def abbr_mapping_func(abbr):
    print("Abbreviation: " + abbr)
    # get all names pointing to that abbreviation
    names = authors[authors.abbreviation == abbr].name
    certainties = authors[authors.abbreviation == abbr][["name", "certainty"]].groupby("name").agg({"certainty": "mean"}).reset_index()
    frequencies = names.value_counts(normalize=True)
    # merge certainties and frequencies on name
    merged = pd.merge(certainties, frequencies, on="name")
    merged["score"] = merged.certainty * merged.proportion
    print(merged)
    return abbr
    
# loop through names
authors.head(20)["abbreviation"].apply(mapping_func)

Abbreviation: kub
             name certainty  proportion     score
0  kai-uwe brandt  0.810345         1.0  0.810345
Abbreviation: jap
        name certainty  proportion score
0  jan peter       0.6         1.0   0.6
Abbreviation: maf
                  name certainty  proportion     score
0  magdalena froehlich   0.89771    0.215993  0.193899
1      manfred lüttich       0.6    0.081616  0.048969
2     michael frömmert  0.751442    0.687552  0.516655
3    märz frank pfütze       0.6    0.014839  0.008904
Abbreviation: ht
          name certainty  proportion score
0  heiko trebs       0.8         1.0   0.8
Abbreviation: art
              name certainty  proportion     score
0  andreas tappert  0.685579         1.0  0.685579
Abbreviation: kub
             name certainty  proportion     score
0  kai-uwe brandt  0.810345         1.0  0.810345
Abbreviation: ar
               name certainty  proportion     score
0    andrea richter  0.888889    0.136364  0.121212
1  angelika raulien       0

id
1     kub
2     jap
3     maf
4      ht
5     art
6     kub
7      ar
8     ast
9     LVZ
10     nt
Name: abbreviation, dtype: object

### Potential Final mapping of a name to an abbreviation
Test if significant shares of name to same abbreviation exist. If theres is a dominant share, assign this as a final mapping

In [ ]:
def name_mapping_func(name):
    print("Name: " + name)
    # get all names pointing to that abbreviation
    abbreviations = authors[authors.name == name].abbreviation
    certainties = authors[authors.name == name][["abbreviation", "certainty"]].groupby("abbreviation").agg({"certainty": "mean"}).reset_index()
    frequencies = abbreviations.value_counts(normalize=True)
    # merge certainties and frequencies on name
    merged = pd.merge(certainties, frequencies, on="abbreviation")
    print(merged)
    return name

# loop through names
authors.head(5)["name"].apply(lambda row: name_mapping_func(row))